<a href="https://colab.research.google.com/github/varun290101/QSTP/blob/main/QSTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
train_df = pd.read_csv("SNLI_TrainSet.csv")
valid_df = pd.read_csv("SNLI_ValidationSet.csv")

In [3]:
train_df.drop(['Unnamed: 0'],axis=1,inplace=True)
valid_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [4]:
train_df=train_df[train_df.similarity != "-"]
valid_df=valid_df[valid_df.similarity != "-"]


In [5]:
train_df=train_df.reset_index(drop=True)
valid_df=valid_df.reset_index(drop=True)

In [6]:
len(train_df),len(valid_df)

(49947, 9842)

In [7]:
k=[]
for i in range(len(train_df)):
  if(train_df['similarity'][i]=='neutral'):
    k.append(0)
  elif(train_df['similarity'][i]=='contradiction'):
    k.append(1)
  else:
    k.append(2)
k1=[]
for i in range(len(valid_df)):
  if(valid_df['similarity'][i]=='neutral'):
    k1.append(0)
  elif(valid_df['similarity'][i]=='contradiction'):
    k1.append(1)
  else:
    k1.append(2)

In [8]:
import numpy as np
y=np.array(k)
y1=np.array(k1)

In [9]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 34.5 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 636 kB 48.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [10]:
from transformers import TFDistilBertModel
from transformers import DistilBertTokenizer

In [11]:
t=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

In [12]:
m=TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [13]:
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [14]:
from keras.models import Model
from keras.layers import Input,Dense,Bidirectional,Dropout,GlobalAveragePooling1D
a=Input(shape=(88,),dtype='int32')
b=Input(shape=(88,),dtype='int32')
emb=m(a,attention_mask=b)[0]
p=Bidirectional(CuDNNLSTM(50,return_sequences=True))(emb)
p=GlobalAveragePooling1D()(p)
p=Dense(100,activation='relu')(p)
p=Dropout(0.1)(p)
out=Dense(64,activation='relu')(p)
m1=Model(inputs=[a,b],outputs=out)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [15]:
m.trainable=False

In [16]:
m1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 66362880    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 88, 100)      328000      tf_distil_bert_model[0][0]   

In [17]:
from keras.layers import Subtract
k1=Input(shape=(88,),dtype='int32')
k2=Input(shape=(88,),dtype='int32')
k3=Input(shape=(88,),dtype='int32')
k4=Input(shape=(88,),dtype='int32')
out1=Subtract()([m1([k1,k2]),m1([k3,k4])])
out1=Dense(100,activation='relu')(out1)
out1=Dropout(0.2)(out1)
out1=Dense(3,activation='softmax')(out1)
model=Model(inputs=[k1,k2,k3,k4],outputs=out1)

In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 88)]         0                                            
____________________________________________________________________________________________

In [19]:
p=train_df['sentence1'].to_list()
p1=train_df['sentence2'].to_list()

In [20]:
d=t(p,max_length=88,padding=True,truncation=True,pad_to_max_length=True)
d1=t(p1,max_length=88,padding=True,truncation=True,pad_to_max_length=True)

In [21]:
q1=np.array(d['input_ids'])
q2=np.array(d['attention_mask'])
q4=np.array(d1['input_ids'])
q5=np.array(d1['attention_mask'])

In [22]:
p2=valid_df['sentence1'].to_list()
p3=valid_df['sentence2'].to_list()

In [23]:
d2=t(p2,max_length=88,padding=True,truncation=True,pad_to_max_length=True)
d3=t(p3,max_length=88,padding=True,truncation=True,pad_to_max_length=True)

In [24]:
q7=np.array(d2['input_ids'])
q8=np.array(d2['attention_mask'])
q10=np.array(d3['input_ids'])
q11=np.array(d3['attention_mask'])

In [25]:
q7.shape

(9842, 65)

In [26]:
num=np.zeros((9842,23))

In [27]:
q7=np.concatenate((q7,num),axis=1)
q8=np.concatenate((q8,num),axis=1)

In [28]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 88)]         0                                            
____________________________________________________________________________________________

In [29]:
import keras
import tensorflow as tf
model.compile(optimizer=tf.optimizers.Adam(6e-5,amsgrad=True),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit([q1,q2,q4,q5],y,epochs=1,batch_size=32,validation_data=([q7,q8,q10,q11],y1))

1561/1561 [==============================] - 805s 502ms/step - loss: 0.9764 - accuracy: 0.5254 - val_loss: 0.8801 - val_accuracy: 0.6020


In [30]:
res=model.predict([q7,q8,q10,q11])

In [31]:
res

array([[0.46322003, 0.49072447, 0.04605544],
       [0.16023085, 0.2840191 , 0.55575   ],
       [0.28611135, 0.6524226 , 0.061466  ],
       ...,
       [0.15275228, 0.20382875, 0.64341897],
       [0.19266568, 0.3577601 , 0.44957417],
       [0.67391217, 0.23580548, 0.09028237]], dtype=float32)

In [32]:
r=np.argmax(res,axis=1)

In [33]:
from sklearn.metrics import classification_report
print(classification_report(y1,r,digits=3))

              precision    recall  f1-score   support

           0      0.648     0.553     0.597      3235
           1      0.576     0.565     0.571      3278
           2      0.591     0.686     0.635      3329

    accuracy                          0.602      9842
   macro avg      0.605     0.601     0.601      9842
weighted avg      0.605     0.602     0.601      9842



In [34]:
sen='you should calm down'
sen1='the world is round'
di=t(sen)
di1=t(sen1)

In [35]:
v=np.array(di['input_ids'])
v1=np.array(di['attention_mask'])
v2=np.array(di1['input_ids'])
v3=np.array(di1['attention_mask'])

In [36]:
v2

array([ 101, 1996, 2088, 2003, 2461,  102])

In [37]:
num=np.zeros((88-len(v)))
num1=np.zeros((88-len(v2)))
v=np.concatenate((v,num))
v1=np.concatenate((v1,num))
v2=np.concatenate((v2,num1))
v3=np.concatenate((v3,num1))

In [38]:
v2.shape

(88,)

In [39]:
res=model.predict([[v.reshape(1,-1),v1.reshape(1,-1),v2.reshape(1,-1),v3.reshape(1,-1)]])

In [40]:
res

array([[0.3704071 , 0.19546032, 0.4341326 ]], dtype=float32)